### Prerequisites

#### Install Libraries

In [ ]:
!pip install textworld ; \
cd /usr/local/lib/python3.7/dist-packages/ && \
rm -rf Diploma_Temp -v ; \
rm -rf textworld/Created_Games -v ; \
rm -rf textworld/MyPy -v ; \
git clone https://github.com/AthanasiosPetsanis/Diploma_Temp  && \
mv Diploma_Temp/TextWorld/Created_Games textworld/ -v && \
mv Diploma_Temp/TextWorld/MyPy textworld/ -v

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.7 MB 13.9 MB/s 
     |████████████████████████████████| 85 kB 4.8 MB/s 
     |████████████████████████████████| 1.2 MB 57.2 MB/s 
ERROR: Operation cancelled by user
Cloning into 'Diploma_Temp'...
remote: Enumerating objects: 450, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 450 (delta 67), reused 91 (delta 26), pack-reused 216
Receiving objects: 100% (450/450), 1.10 MiB | 6.50 MiB/s, done.
Resolving deltas: 100% (149/149), done.
renamed 'Diploma_Temp/TextWorld/Created_Games' -> 'textworld/'
renamed 'Diploma_Temp/TextWorld/MyPy' -> 'textworld/MyPy'


#### Import Libraries

In [ ]:
# This imports all the necessary libraries from inside Q_agent.py
from textworld.MyPy.Q_agent import *

ModuleNotFoundError: ignored

### Params

In [ ]:
# env_names = ['Easy', 'Easy_2', 'Medium', 'Medium_2']
# env_names = ['Medium_2']
env_names = ['Hard']
algo = 'ppo'
graph_points = 1000
frames = graph_points * 2048
recurrence = 8
action_courses = {}
user_inputs = {}

In [ ]:
envs = []
for env_name in env_names:
    envs.append('Sparse_' + env_name)
    envs.append('Dense_' + env_name)

user_inputs['Easy'] = ['apple', 'livingroom', 'table']
user_inputs['Easy_2'] = ['apple', 'livingroom', 'table']
user_inputs['Medium'] = ['apple', 'livingroom', 'table']
user_inputs['Medium_2'] = ['apple', 'livingroom', 'table']
user_inputs['Hard'] = ['apple', 'livingroom', 'table']


In [ ]:
print(action_courses)

### TextWorld Training

In [ ]:
for env_name in env_names:
    
    # Load env
    game_file = '/usr/local/lib/python3.7/dist-packages/textworld/Created_Games/' + env_name + '.ulx'
    env, obs, infos = load_game(game_file=game_file)

    # Make agent
    agent = Q_agent(env)

    # Train
    agent.reset()
    agent.train(max_epochs=5, max_eps=200, title=env_name, min_expl=0.02, user_input=user_inputs[env_name])
    # agent.train(max_epochs=5, max_eps=100, title=env_name, min_expl=0.05, manual=True)

    # Test
    action_courses[env_name] = agent.test()

### Post TextWorld Process

In [ ]:
# !cd /usr/local/lib/python3.7/dist-packages/ && \
# rm -rf Diploma_Temp -v

!pip3 install gym-minigrid ; \
cd /usr/local/lib/python3.7/dist-packages/ && \
git clone https://github.com/lcswillems/rl-starter-files ; \
git clone https://github.com/lcswillems/torch-ac ; \
mv Diploma_Temp/MiniGrid/__init__.py gym_minigrid/envs/ -v ; \
mv Diploma_Temp/MiniGrid/MyEnvs.py gym_minigrid/envs/ -v ; \
mv Diploma_Temp/MiniGrid/minigrid.py gym_minigrid/ -v ; \
cd /usr/local/lib/python3.7/dist-packages/rl-starter-files/ && \
pip3 install -r requirements.txt ; \

In [ ]:
import pickle

def filter_actions(action_courses):
    rmv_actions = ['go east', 'go west', 'go north', 'go south']
    for action_course, actions_taken in action_courses.items():
        for sentence in actions_taken:
            if 'unlock' in sentence:
                action_courses[action_course].remove(sentence)
        action_courses[action_course] = list(filter(lambda action: action not in rmv_actions, actions_taken))
    return action_courses

def reset(envs):
    for env_name in envs:
        !cd /usr/local/lib/python3.7/dist-packages/rl-starter-files/storage && \
        rm -rf {env_name} -v

action_courses = filter_actions(action_courses)

with open('/usr/local/lib/python3.7/dist-packages/rl-starter-files/storage/action_courses', 'wb') as fp:
    pickle.dump(action_courses, fp)

In [ ]:
print(action_courses)

### MiniGrid Training

In [ ]:
!cd /usr/local/lib/python3.7/dist-packages/gym_minigrid/envs/ && \
python3 MyEnvs.py

import time
reset(envs)
envs = ['Dense_Hard']
for env_name in envs: 
    env_dir = 'MiniGrid-' + env_name + '-v0'
    !cd /usr/local/lib/python3.7/dist-packages/rl-starter-files/ && \
    python3 -m scripts.train --algo {algo} --env {env_dir} --frames {frames} --model {env_name} --recurrence {recurrence}


### Graphs

In [ ]:
import csv
import matplotlib.pyplot as plt

def data_extract(envs, data_label):

    storage_data = []
    new_dir_names = []
    
    for env_name in envs:
        file_name = '/usr/local/lib/python3.7/dist-packages/rl-starter-files/storage/' + env_name + '/log.csv'
        file = open(file_name)
        file = csv.reader(file)
        header = next(file)

        data = []
        for d in file:
            data.append(d)
        for i in range(len(header)):
            if header[i] == data_label: r_index = i

        data_hist = []
        for epoch in range(len(data)):
            sample = data[epoch][r_index]
            if sample != data_label:
                data_hist.append(round(float(sample), 5))
        
        storage_data.append(data_hist)
    return storage_data

# data_label = 'rreturn_mean'
data_label = 'rreturn_mean'
data = data_extract(envs, data_label)

In [ ]:
# Simple Representation
for i in np.arange(0,len(data),2):
    fig = plt.figure(figsize=(6,4),facecolor='skyblue', edgecolor='black',dpi=100)

    title = env_names[int(i/2)]
    ax = fig.add_axes([0,0,1,1], projection='rectilinear',
    xlabel = 'Something',
    ylabel = 'Mean Episode Return', title=title, facecolor='w')

    plt.plot(data[i], color='blue', label='Sparse Rewarding')
    # plt.plot(data[i+1], color='r', label='Dense Rewarding')
    plt.legend()

In [ ]:
print(envs)

In [ ]:
!cd /usr/local/lib/python3.7/dist-packages/rl-starter-files/storage/{env_name}/ && ls